In [1]:
import gc
import matplotlib.pyplot as plt
from nnom import *
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.layers import *
from keras.activations import *
from keras import *

2024-11-06 23:37:32.458662: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-06 23:37:32.458742: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-06 23:37:32.460113: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-06 23:37:32.589870: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### 一、加载数据

In [2]:
motion_name = ["Circle", "Letter_L", "Letter_M", "Letter_R", "Letter_W", "NoMotion", "Triangle", "Upanddown"]
data_per_size = 50
class_num = len(motion_name)
# 全部数据
total_data = []
# 标签数组
total_y = np.array([j for j in range(class_num) for i in range(data_per_size)])

# 获取标签的唯一值并排序
total_labels = np.unique(total_y)

# 创建独热编码矩阵
total_labels = np.eye(len(total_labels))[total_y]

for mo_name in motion_name:
    for i in range(data_per_size):
        data = np.loadtxt(f"/home/whiz/Projects/Motion_classification/Dataset/{mo_name}/{mo_name}_{i}.txt", delimiter=',')
        total_data.append(data[ : , -3 :])
        
total_data = np.array(total_data)

gc.collect()

0

### 二、分割数据集

### 三、创建模型

In [3]:
def inception_model():
    # 定义输入层
    inputs = Input(shape=(200, 3))

    x = Conv1D(32, kernel_size=(9), strides=(2), padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    x = ReLU()(x)
    x = MaxPool1D(2, strides=2)(x)

    # inception - 1
    x1 = Conv1D(32, kernel_size=(5), strides=(1), padding="same")(x)
    x1 = BatchNormalization()(x1)
    x1 = Dropout(0.2)(x1)
    x1 = ReLU()(x1)
    x1 = MaxPool1D(2, strides=2)(x1)

    # inception - 2
    x2 = Conv1D(32, kernel_size=(3), strides=(1), padding="same")(x)
    x2 = BatchNormalization()(x2)
    x2 = Dropout(0.2)(x2)
    x2 = ReLU()(x2)
    x2 = MaxPool1D(2, strides=2)(x2)

    # inception - 3
    x3 = MaxPool1D(2, strides=2)(x)
    x3 = Dropout(0.2)(x3)

    # concate all inception layers
    x = concatenate([ x1, x2,x3], axis=-1)

    #conclusion
    x = Conv1D(48, kernel_size=(3), strides=(1), padding="same")(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    #x = MaxPool1D(2, strides=2)(x)
    x = Dropout(0.2)(x)

    # our netowrk is not that deep, so a hidden fully connected layer is introduce
    x = Flatten()(x)
    x = Dense(64)(x)
    x = Dropout(0.2)(x)
    x = ReLU()(x)
    x = Dense(class_num)(x)
    outputs = layers.Softmax()(x)

    # 创建模型
    model = Model(inputs=inputs, outputs=outputs)
    # 编译模型，优化器使用adam，损失函数使用交叉熵损失函数，评估标准为准确率  
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    
    return model

def rnn_model():
    inputs = Input(shape=(200, 3))
    x = Conv1D(9, kernel_size=(9), strides=(3), padding='same')(inputs)
    x = BatchNormalization()(x)

    # you can use either of the format below.
    # x = RNN(SimpleRNNCell(16), return_sequences=True)(x)
    # x = SimpleRNN(16, return_sequences=True)(x)

    # x2 = RNN(LSTMCell(32), return_sequences=True)(x)
    # x1 = LSTM(32, return_sequences=True, go_backwards=True)(x)
    # x = concatenate([x1, x2], axis=-1)
    #
    # # Bidirectional with concatenate. (not working yet)
    # x1 = RNN(GRUCell(16), return_sequences=True)(x)
    # x2 = GRU(16, return_sequences=True, go_backwards=True)(x)
    # x = concatenate([x1, x2], axis=-1)


    # Bidirectional with concatenate. (not working yet)

    # x1 = LSTM(32, return_sequences=True)(x)
    # x2 = LSTM(32, return_sequences=True, go_backwards=True)(x)
    # x = add([x1, x2])
    x = LSTM(32, return_sequences=True)(x)
    x1 = GRU(32, return_sequences=True)(x)
    x2 = GRU(32, return_sequences=True, go_backwards=True)(x)
    x = concatenate([x1, x2], axis=-1)

    x = GRU(32, return_sequences=True)(x)
    x = GRU(32, return_sequences=True)(x)

    x = Flatten()(x)
    x = Dense(64)(x)
    x = ReLU()(x)
    x = Dropout(0.2)(x)
    x = Dense(class_num)(x)
    predictions = Softmax()(x)

    model = Model(inputs=inputs, outputs=predictions)

    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    model.summary()
    
    return model

def train():
    # 1. 分割为训练集和测试集
    X_train, X_test, y_train, y_test = train_test_split(total_data, total_labels, test_size=0.2, shuffle=True, stratify=total_labels)

    # 2. 进一步分割训练集为训练集和验证集
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, shuffle=True, stratify=y_train)


    model = inception_model()
    # model = rnn_model()
    
    # 训练模型  
    history = model.fit(X_train, y_train, epochs=100, batch_size=50, validation_data=(X_val, y_val), verbose=2)

    # 评估模型
    loss, accuracy = model.evaluate(X_test, y_test)
    
    return model, loss, accuracy, X_test

for i in range(20):
    model, loss, accuracy, X_test = train()
    if accuracy > 0.85:
        from keras.models import load_model
        # 保存模型
        from keras.models import save_model
        save_model_name = f"./save_model/model_{i}_loss_{loss}_acc_{accuracy}"
        save_model(model, save_model_name + ".keras")
        
        L_model = load_model(save_model_name + ".keras")

        # 假设generate_model函数已经定义在nnom模块中
        generate_model(L_model, X_test, name = save_model_name + ".h")
    del model
    

2024-11-06 23:38:02.023541: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-06 23:38:02.266181: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-06 23:38:02.266248: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-06 23:38:02.269059: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-06 23:38:02.269116: I tensorflow/compile

Epoch 1/100


2024-11-06 23:38:04.371880: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700
2024-11-06 23:38:06.052016: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fa22755da60 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-11-06 23:38:06.052061: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Compute Capability 8.6
2024-11-06 23:38:06.058515: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-11-06 23:38:06.132237: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


6/6 - 6s - loss: 1.8205 - accuracy: 0.4023 - val_loss: 1.6757 - val_accuracy: 0.4375 - 6s/epoch - 1s/step
Epoch 2/100
6/6 - 0s - loss: 0.8011 - accuracy: 0.8086 - val_loss: 1.7373 - val_accuracy: 0.4531 - 115ms/epoch - 19ms/step
Epoch 3/100
6/6 - 0s - loss: 0.5172 - accuracy: 0.8789 - val_loss: 1.1819 - val_accuracy: 0.6406 - 112ms/epoch - 19ms/step
Epoch 4/100
6/6 - 0s - loss: 0.3205 - accuracy: 0.9180 - val_loss: 0.6685 - val_accuracy: 0.7969 - 116ms/epoch - 19ms/step
Epoch 5/100
6/6 - 0s - loss: 0.1834 - accuracy: 0.9688 - val_loss: 0.6957 - val_accuracy: 0.7656 - 112ms/epoch - 19ms/step
Epoch 6/100
6/6 - 0s - loss: 0.1582 - accuracy: 0.9570 - val_loss: 0.7549 - val_accuracy: 0.7031 - 124ms/epoch - 21ms/step
Epoch 7/100
6/6 - 0s - loss: 0.1236 - accuracy: 0.9609 - val_loss: 0.6397 - val_accuracy: 0.7344 - 115ms/epoch - 19ms/step
Epoch 8/100
6/6 - 0s - loss: 0.1106 - accuracy: 0.9766 - val_loss: 0.4451 - val_accuracy: 0.8438 - 130ms/epoch - 22ms/step
Epoch 9/100
6/6 - 0s - loss: 0.11

KeyboardInterrupt: 

In [25]:
# 保存模型
from keras.models import save_model
save_model_name = "best_model"
save_model(model, save_model_name + ".keras")

In [ ]:
from keras.models import load_model
L_model = load_model(save_model_name + ".keras")

loss, accuracy = L_model.evaluate(X_test, y_test)

# 假设generate_model函数已经定义在nnom模块中
generate_model(L_model, X_test, name = save_model_name + ".h")